In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Read the housing data into pandas dataframe
Point2_data =  pd.read_csv('/users/aeternus/Desktop/Housing-market-in-Canada/Point2_data_Canada_No_NaN.csv')


# Create a dash application
app = JupyterDash(__name__)

Regions = set(Point2_data['Region'].values)
Regions.add('Canada')
# Build dash app layout
app.layout = html.Div(children=[ html.H1('Canada Housing Market Statistics',
                                style={'textAlign': 'center', 'color': '#503D36', 
                                'font-size': 30}),
                                html.Br(),
                                dcc.Dropdown(id='Region',
                                                    value='Canada',
                                                    options=
                                                    [{'value': x, 'label':x} for x in Regions],
                                                    placeholder='Select a Region here',
                                                    searchable=True),
                                html.Br(),
                                html.Div([
                                        html.Div(dcc.Graph(id='bedroom-plot')),
                                        html.Div(dcc.Graph(id='bathroom-plot'))
                                ], style={'display': 'flex'}),

                                html.Div([
                                        html.Div(dcc.Graph(id='combo-plot'))
                                ], style={'display': 'flex'}),
                                
                                html.Div([
                                        html.Div(dcc.Graph(id='price-plot'))
                                ], style={'display': 'flex'}),
                                

                                ])

# Callback decorator
@app.callback( [
               Output(component_id='bedroom-plot', component_property='figure'),
               Output(component_id='bathroom-plot', component_property='figure'),
               Output(component_id='combo-plot', component_property='figure'), 
               Output(component_id='price-plot', component_property='figure'),
               ],
               Input(component_id='Region', component_property='value'))
# Computation to callback function and return graph

def get_graph(region):
    
    if region == "Canada":
        bed_counts = Point2_data['Beds'].value_counts().reset_index()
        bath_counts = Point2_data['Bath'].value_counts().reset_index()
        df_buble = pd.DataFrame({'Counts':(Point2_data[['Beds','Bath']].groupby(['Beds','Bath']).size())}).reset_index()
        df_price = Point2_data[['Beds','Bath','Price']].groupby(['Beds','Bath']).mean().reset_index()

    else:
        df =  Point2_data[Point2_data['Region']==str(region)]
        bed_counts = df['Beds'].value_counts().reset_index()
        bath_counts = df['Bath'].value_counts().reset_index()
        df_buble = pd.DataFrame({'Counts':(df[['Beds','Bath']].groupby(['Beds','Bath']).size())}).reset_index()
        df_price = df[['Beds','Bath','Price']].groupby(['Beds','Bath']).mean().reset_index()
    
    # Bar plot for bedrooms
    bed_counts_fig = px.bar(bed_counts, x='index', y='Beds', title= 'Housing market with Bedrooms',
                           labels={"index": "Number of Bedrooms","Beds": "Number of houses"})
    bed_counts_fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 1, dtick = 1))
    
    # Bar plot for bathrooms
    bath_counts_fig = px.bar(bath_counts, x='index', y='Bath', title='Housing market with Bathrooms',
                            labels={"index": "Number of Bathrooms","Bath": "Number of houses"})
    bath_counts_fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 1, dtick = 1))
    
    # Buble plot for counts of combos
    norm = (df_buble['Counts'] - df_buble['Counts'].min()) / (df_buble['Counts'].max() - df_buble['Counts'].min())
    df_buble_fig = px.scatter(df_buble, x="Beds", y="Bath",size="Counts", log_x=True, size_max=60)
    df_buble_fig.update_layout(
    xaxis_title="Number of Bedrooms", yaxis_title="Number of Bathrooms",
    xaxis = dict(tickmode = 'array', tickvals = np.arange(0, 17, step=1), ticktext = np.arange(0, 17, step=1)),
    yaxis = dict(tickmode = 'array', tickvals = np.arange(0, 17, step=1), ticktext = np.arange(0, 17, step=1)),width=1000,height=800)
    
    # Buble plot for price of combos
    norm2 = (df_price['Price'] - df_price['Price'].min()) / (df_price['Price'].max() - df_price['Price'].min())
    df_price_fig = go.Figure(data=[go.Scatter(x=df_price["Beds"], y=df_price["Bath"],text=df_price["Price"].astype(int),
                                               mode='markers', marker_size=norm2*100)])
    df_price_fig.update_layout(
    xaxis_title="Number of Bedrooms", yaxis_title="Number of Bathrooms",
    xaxis = dict(tickmode = 'array', tickvals = np.arange(0, 17, step=1), ticktext = np.arange(0, 17, step=1)),
    yaxis = dict(tickmode = 'linear',tick0 = 1, dtick = 1),width=1000,height=600)
    return[bed_counts_fig, bath_counts_fig, df_buble_fig, df_price_fig]
# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost")